# Summarize a Website Using the Llama v3.2 Model

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [5]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
def summarize_via_ollama(url):
    website = Website(url)    
    messages = messages_for(website)

    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }
    
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
       
    return response.json()['message']['content']

In [8]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary_via_ollama(url):
    summary = summarize_via_ollama(url)
    display(Markdown(summary))

In [9]:
display_summary_via_ollama("https://edwarddonner.com")

# Summary of Website: Edward Donner's Home Page

## Overview

This website belongs to Edward Donner, the co-founder and CTO of Nebula.io. The homepage is a personal introduction to his work in AI and technology.

## Latest News/Announcements

- **2025 AI Executive Briefing**: A scheduled event on May 18, 2025
- **The Complete Agentic AI Engineering Course**: Announced for January 23, 2025
- **LLM Workshop – Hands-on with Agents – resources**: Available as a resource on December 21, 2024

## Personal Details

Edward Donner shares his personal interests in coding, electronic music production, and DJing. He also mentions his previous experience as the founder of AI startup untapt.

## Contact Information

- Email: [ed at] edwarddonner [dot] com
- Website: www.edwarddonner.com
- Social Media:
  - LinkedIn
  - Twitter
  - Facebook